In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, accuracy_score
)
from xgboost import XGBClassifier

In [12]:
# Load dataset
df = pd.read_csv("/content/drive/MyDrive/CIS_508/team_project/training_data_2022-12-17-pdb-intersect-pisces_pc30_r2.5.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CIS_508/team_project/training_data_2022-12-17-pdb-intersect-pisces_pc30_r2.5.csv'

### Compute helix/sheet/coil fractions from sst3

In [ ]:
def compute_sst3(sst3_string):
  total = len(sst3_string)
  if total == 0:
    return 0,0,0

  helix = sst3_string.count("H")/ total
  sheet = sst3_string.count("E")/ total
  coil = sst3_string.count("C")/ total

  return helix, sheet, coil

In [ ]:
df[['helix_frac', 'sheet_frac', 'coil_frac']] = df['sst3'].apply(lambda x: pd.Series(compute_sst3(x)))
df.head()

### Compute data-driven thresholds

In [ ]:
helix_thresh = df["helix_frac"].median()
sheet_thresh = df["sheet_frac"].median()
coil_thresh = df["coil_frac"].median()
rfac_thresh   = df['rfac'].median()
freer_thresh  = df['freerfac'].median()

print("Helix Thresh: ", helix_thresh)
print("Sheet Thresh: ", sheet_thresh)
print("Coil Thresh: ", coil_thresh)
print("rfac Thresh: ", rfac_thresh)
print("freerfac Thresh: ", freer_thresh)

### Compute stability score

In [ ]:
df['stability_score'] = (
      (df['helix_frac'] > helix_thresh).astype(int)
    + (df['sheet_frac'] > sheet_thresh).astype(int)
    + (df['coil_frac']  < coil_thresh).astype(int)
    + (df['rfac']       < rfac_thresh).astype(int)
    + (df['freerfac']   < freer_thresh).astype(int)
)

df['is_stable'] = (df['stability_score'] >= 3).astype(int)

df[['stability_score', 'is_stable']].head()


In [ ]:
df.head()

In [ ]:
df.to_csv("/content/drive/MyDrive/CIS_508/team_project/training_data_preprocessed", index = False)

### EDA

### Distribution plots for helix/sheet/coil %

In [ ]:
plt.figure(figsize=(15, 4))
for i, col in enumerate(['helix_frac', 'sheet_frac', 'coil_frac']):
    plt.subplot(1, 3, i+1)
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f"Distribution of {col}")
plt.show()

### rfac and freerfac distributions

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
sns.histplot(df['rfac'], kde=True, bins=30, color="teal")
plt.title("R-factor Distribution")

plt.subplot(1, 2, 2)
sns.histplot(df['freerfac'], kde=True, bins=30, color="coral")
plt.title("Free R-factor Distribution")

plt.show()


### Correlation heatmap

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(df[['helix_frac','sheet_frac','coil_frac','rfac','freerfac','is_stable']].corr(),
            annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()


### Class balance plot

In [ ]:
sns.countplot(data=df, x='is_stable', palette="viridis")
plt.title("Stable vs Unstable Distribution")
plt.show()


### Sequence Feature Engineering

In [ ]:
AA = list("ACDEFGHIKLMNPQRSTVWY")

def aa_features(seq):
    length = len(seq)
    counts = {aa: seq.count(aa) / length for aa in AA}
    counts["length"] = length
    return pd.Series(counts)

aa_df = df['seq'].apply(aa_features)

In [ ]:
features = pd.concat([aa_df, df[['is_stable']]], axis=1)
features.head()


### Train-Test Split

In [ ]:
X = features.drop("is_stable", axis=1)
y = features["is_stable"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
clf = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced",
    random_state=42
)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1]

### Evaluation

### Classification Report

In [ ]:
print(classification_report(y_test, y_pred))

### Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


### ROC Curve and AUC

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_prob)
auc = roc_auc_score(y_test, y_prob)

plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
plt.plot([0,1],[0,1],'--',color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()


### Model-1 Evaluation Summary
The classifier achieved an overall accuracy of 60%, with precision, recall, and F1-scores hovering around 0.58–0.63 for both stable and unstable classes. This indicates that the model is learning meaningful patterns from amino-acid composition but remains limited by the simplicity of the input features.

The confusion matrix shows a relatively balanced error distribution across classes. Both stability states experience similar degrees of misclassification, suggesting no strong model bias and reflecting the inherent difficulty of predicting structural stability from high-level sequence features alone.

The ROC curve yields an AUC of 0.621, confirming that the classifier performs better than random guessing, yet the predictive signal remains modest. This is expected because protein stability is shaped by complex structural and physicochemical interactions that are not fully captured by composition-based features.

Overall, the model serves as a solid baseline for sequence-only stability prediction, demonstrating that the engineered stability label derived from structural properties contains learnable signal, while also highlighting the need for richer sequence embeddings or structural descriptors to achieve higher predictive performance.

### XGBoost

In [ ]:
xgb_clf = XGBClassifier(
    n_estimators=400,          # number of trees
    max_depth=5,              # tree depth
    learning_rate=0.05,       # step size shrinkage
    subsample=0.8,            # row sampling
    colsample_bytree=0.8,     # feature sampling
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

xgb_clf.fit(X_train, y_train)

### Predictions and metrics

In [ ]:
y_pred_xgb = xgb_clf.predict(X_test)
y_prob_xgb = xgb_clf.predict_proba(X_test)[:, 1]

print("=== XGBoost Classification Report ===")
print(classification_report(y_test, y_pred_xgb))

### Confusion matrix

In [ ]:
cm_xgb = confusion_matrix(y_test, y_pred_xgb)

plt.figure(figsize=(5,4))
sns.heatmap(cm_xgb, annot=True, fmt="d", cmap="Blues")
plt.title("Model 2: XGBoost - Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

### ROC curve and AUC

In [ ]:
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_prob_xgb)
auc_xgb = roc_auc_score(y_test, y_prob_xgb)

plt.figure(figsize=(6,5))
plt.plot(fpr_xgb, tpr_xgb, label=f"AUC = {auc_xgb:.3f}")
plt.plot([0,1], [0,1], "--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Model 2: XGBoost - ROC Curve")
plt.legend()
plt.show()

print(f"XGBoost AUC: {auc_xgb:.3f}")

### Mutation Prediction
Isolation Forest

In [ ]:
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

### Build structural feature matrix

In [ ]:
# Structural / crystallographic feature set
struct_cols = [
    "helix_frac",
    "sheet_frac",
    "coil_frac",
    "rfac",
    "freerfac",
    "resol",
    "len_x",
    "len_y"
]

# Subset the dataframe to these features
X_struct = df[struct_cols].copy()

# 1) Impute missing values (freerfac etc.)
imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X_struct)

# 2) Scale features (important for KMeans + IsolationForest)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Wrap back into a DataFrame to preserve column names & index
X_struct_scaled = pd.DataFrame(
    X_scaled,
    columns=struct_cols,
    index=df.index
)

X_struct_scaled.head()

### Cluster proteins by structural profile

In [ ]:
# Choose number of clusters (you can tune this later if you want)
n_clusters = 4

kmeans = KMeans(
    n_clusters=n_clusters,
    random_state=42,
    n_init=10
)

cluster_labels = kmeans.fit_predict(X_struct_scaled)

# Attach cluster labels to df
df["struct_cluster"] = cluster_labels

df["struct_cluster"].value_counts()


### Run Isolation Forest within each cluster

In [ ]:
# Containers to store per-protein anomaly labels and scores
if_labels = pd.Series(index=df.index, dtype="float64")
if_scores = pd.Series(index=df.index, dtype="float64")

# We use the same scaled structural matrix, but split by cluster
unique_clusters = sorted(df["struct_cluster"].unique())

for c in unique_clusters:
    # indices of proteins in this cluster
    idx = df.index[df["struct_cluster"] == c]

    # feature matrix for this cluster (scaled)
    X_c = X_struct_scaled.loc[idx]

    # Skip clusters that are too tiny for a meaningful IF model
    if len(X_c) < 10:
        print(f"Skipping cluster {c} (only {len(X_c)} samples).")
        continue

    # Isolation Forest for this cluster
    iso = IsolationForest(
        n_estimators=200,
        contamination=0.05,  # assumes ~5% outliers per cluster
        random_state=42,
        n_jobs=-1
    )

    iso.fit(X_c)

    # Predictions: 1 = inlier, -1 = outlier
    preds = iso.predict(X_c)
    scores = iso.decision_function(X_c)  # lower score = more anomalous

    if_labels.loc[idx] = preds
    if_scores.loc[idx] = scores

# Attach to df
df["if_label"] = if_labels
df["if_score"] = if_scores

# Binary flag: 1 = outlier, 0 = normal
df["is_outlier"] = (df["if_label"] == -1).astype("Int64")

df[["struct_cluster", "if_label", "if_score", "is_outlier"]].head()

### Quick sanity checks

In [ ]:
df["is_outlier"].value_counts(dropna=False)

### Outliers vs stability label

In [ ]:
pd.crosstab(df["is_stable"], df["is_outlier"], rownames=["is_stable"], colnames=["is_outlier"])

### Inspect top anomalous proteins

In [ ]:
# Most anomalous proteins (across all clusters)
top_outliers = df[df["is_outlier"] == 1].sort_values("if_score").head(20)

cols_to_show = [
    "pdb_id",
    "chain_code",
    "struct_cluster",
    "if_score",
    "is_stable",
    "helix_frac",
    "sheet_frac",
    "coil_frac",
    "rfac",
    "freerfac",
    "resol",
    "len_x",
    "len_y"
]

top_outliers[cols_to_show]

### Quick visualization

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=df,
    x="helix_frac",
    y="sheet_frac",
    hue="struct_cluster",
    style="is_outlier",
    palette="tab10",
    alpha=0.7
)
plt.title("Structural Clusters with Outliers (Isolation Forest)")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

### Conclusions: Mutation-Impact Outlier Detection Using Cluster-Aware Isolation Forest


The anomaly detection pipeline was designed to identify proteins whose structural features deviate significantly from the expected patterns of their respective structural families. By combining K-Means clustering with Isolation Forest, the approach ensures that outlier detection is contextual—a protein is considered anomalous only relative to proteins with similar secondary-structure compositions, refinement statistics, and crystallographic characteristics.


1. Structural Clustering Establishes Biologically Meaningful Groups

Using K-Means on standardized structural features (helix_frac, sheet_frac, coil_frac, rfac, freerfac, resol, len_x, len_y), the dataset naturally organized into four coherent clusters:

Proteins with high helix content

Sheet-dominant proteins

Coil-rich or disordered proteins

Mixed/heterogeneous structural compositions

Each cluster reflects a consistent structural “signature,” which provides a stable reference distribution for subsequent anomaly detection. This clustering step prevents false positives that arise when structurally distinct proteins are compared globally rather than locally.


2. Isolation Forest Effectively Identifies Structurally Deviant Proteins

Isolation Forest was trained independently within each cluster, enabling detection of proteins that violate the internal structure of that cluster. Approximately 5% of proteins were flagged as outliers, consistent with the contamination setting and validating stable model behavior.

The detected outliers exhibit clear structural abnormalities, including:

Extreme coil dominance (coil_frac = 1.0) with zero helix or sheet content

Very short chain lengths, typical of peptides or unresolved fragments

Unusual refinement statistics (rfac, freerfac) relative to cluster norms

Mid-to-poor resolution structures, increasing structural uncertainty

These deviations strongly align with known signatures of unstable, flexible, mutated, or poorly resolved protein models.

3. Relationship Between Outliers and Stability

The cross-tabulation with the binary is_stable label reveals:

Outliers appear in both stable and unstable classes

A modest enrichment of outliers among unstable proteins

A non-trivial minority of “stable” proteins are still flagged as anomalous due to their structural irregularities

This demonstrates that Isolation Forest provides complementary structural information beyond the stability label. While classification models predict stability directly, the anomaly detector highlights proteins that are structurally inconsistent, even if they are not explicitly labeled unstable. This dual perspective enhances understanding of stability mechanisms and mutation effects.

4. Visual Validation Confirms Biological Interpretability

The 2D cluster plot of helix versus sheet fractions shows:

Clear separation between structural families

Outliers concentrated at the edges of structural space

Anomalous proteins lying in sparsely populated or biologically unusual regions

Patterns consistent with disordered proteins, unresolved models, or mutation-driven structural shifts

This visualization reinforces that the detected outliers are not random artifacts but reflect legitimate deviations in secondary structure composition.

5. Overall Significance of the Method

The cluster-aware Isolation Forest approach successfully uncovers proteins that are likely to represent:

Mutation-induced structural perturbations

Unstable or partially folded conformations

Disordered regions misaligned with cluster norms

Poorly refined crystallographic entries

Structural anomalies not captured by classical stability metrics

The method enhances interpretability and provides a robust framework for downstream biological analysis, such as:

Identifying candidate mutation effects

Assessing structure–stability discrepancies

Flagging proteins for quality control or deeper structural investigation

Summary

The implemented pipeline reliably detects structurally anomalous proteins within biologically meaningful clusters. These anomalies correlate with expected signatures of instability and disorder, supporting the validity of the approach. By integrating clustering with Isolation Forest, the model captures subtle structural deviations that traditional global anomaly methods or stability classifiers alone cannot identify.

In [ ]:
!jupyter nbconvert --to html "Stability and Mutation Prediction (1).ipynb"